In [2]:
import pylab as plt
import pymoc
import xidplus
import numpy as np
%matplotlib inline
from astropy.table import Table, join

In [3]:
import seaborn as sns

This notebook uses all the raw data from the CIGALE predictions and photoz catalogue, maps, PSF and relevant MOCs to create XID+ prior object and relevant tiling scheme

## Read in MOCs
The selection functions required are the main MOC associated with the masterlist. 

In [4]:
Sel_func=pymoc.MOC()
Sel_func.read('../../dmu4/dmu4_sm_GAMA-12/data/holes_GAMA-12_ukidss_k_O16_20180419_MOC.fits')


## Read in CIGALE predictions catalogue

In [39]:
cigale=Table.read('../../dmu28/dmu28_GAMA-12/data/GAMA-12_Ldust_prediction_results.fits')


In [40]:
cigale['id'].name = 'help_id'


In [41]:
cigale

help_id,bayes.dust.luminosity,bayes.dust.luminosity_err,best.omegacam_u,best.omegacam_g,best.suprime_g,best.decam_g,best.gpc1_g,best.suprime_r,best.gpc1_r,best.omegacam_r,best.decam_r,best.gpc1_i,best.omegacam_i,best.suprime_i,best.gpc1_z,best.vista_z,best.suprime_z,best.decam_z,best.gpc1_y,best.suprime_y,best.vista_y,best.ukidss_y,best.ukidss_j,best.vista_j,best.ukidss_h,best.vista_h,best.vista_ks,best.ukidss_k
,,,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy
str28,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
HELP_J112916.236+004455.105,9.65287793768e+37,1.17666784092e+38,0.000980295023014,0.00152945692592,0.00155522672771,0.00156031374351,0.0015811572678,0.00319442648463,0.00323499658544,0.00344452392723,0.00365353497765,0.00506550119885,0.00507429427253,0.00516587092685,0.00571054914042,0.005799151538,0.00589362525503,0.00606803848146,0.00634927742142,0.00641461877416,0.00659292795451,0.00665987676509,0.00809632604272,0.00811067438847,0.0106524893003,0.0106792419785,0.0122238062342,0.0124163888182
HELP_J112916.735+004525.955,4.99604151924e+38,6.01405625316e+38,0.00228961737982,0.00416181870533,0.00425796740958,0.00427614671171,0.00434788511322,0.00959801620012,0.00965743796182,0.0103856772571,0.0110859039015,0.0168555632412,0.0170860313911,0.0175084982515,0.0223159250776,0.0228707747602,0.0235845969178,0.0248448303069,0.027143746692,0.027827712213,0.0299828410087,0.0306283344946,0.0437204990637,0.0439311275299,0.0693150570334,0.0696774039528,0.0906166212251,0.0936494380324
HELP_J112916.868+004538.834,6.74395178932e+35,2.11732503197e+35,0.00797875041146,0.0298080393801,0.0305873658148,0.0307507600584,0.0316232736597,0.0574327371112,0.0577872312662,0.0598883825059,0.0617535017724,0.0808511378306,0.081672435631,0.0833186989953,0.101873791141,0.104195668445,0.106982286305,0.114021556098,0.125815868503,0.12929421768,0.137854161055,0.14010272913,0.163531125537,0.163858604646,0.207051791153,0.208521374093,0.195515497984,0.19135190687
HELP_J112916.993+004528.665,1.06941218163e+37,1.18786921122e+37,0.000958880860804,0.00204094231687,0.00212978078874,0.00215053957163,0.00221212661902,0.00438028344746,0.00439633094758,0.00453091385133,0.00465871777647,0.00602217800415,0.00607213083905,0.0061811261165,0.00718156074182,0.00729981335821,0.00744050683883,0.00781017044537,0.00848844401419,0.00862261078026,0.00902709450123,0.00914284994064,0.012212649293,0.0122324762736,0.0145243616455,0.0145698906336,0.0179420683456,0.0178088588752
HELP_J112917.687+004454.217,8.27142268505e+37,5.40913585177e+37,0.00137585337232,0.00538198184203,0.0058148696356,0.00590330344897,0.00614237388744,0.0182970759978,0.0183700985544,0.0194974307305,0.0205248944548,0.0319632835879,0.0324309746237,0.0334546451974,0.0437975562519,0.0447575259366,0.0461367954266,0.0498327120644,0.0556160601284,0.0580288181608,0.0628588008841,0.0640299734127,0.0980593652049,0.0984818456491,0.137654757191,0.138522057534,0.196027629972,0.198087233649
HELP_J112917.704+004533.215,2.62197498446e+36,1.3701873132e+36,0.016534572991,0.0840618775116,0.0873091714541,0.0879454585475,0.0909989571154,0.195372872856,0.196313152927,0.205333356537,0.214011374115,0.30729033966,0.313252209594,0.321932819252,0.423012204598,0.435123206332,0.446370918813,0.483403907101,0.546884818768,0.566406555256,0.614831497502,0.626988793227,0.783608565337,0.785897174058,1.1039665533,1.1128944361,1.08867447396,1.06836625457
HELP_J112918.031+004449.789,2.47229097952e+37,1.38352426361e+37,0.012063642223,0.0116270626371,0.0117979015648,0.0118222894812,0.0118366856847,0.0142852318447,0.0142829495391,0.014203194741,0.0141328553595,0.0132481827668,0.0133045894053,0.0132304404813,0.0127280861781,0.0126761603511,0.0125914341498,0.0126227125551,0.0125912927721,0.0125654024821,0.0126517230037,0.01

In [48]:
for i in range(0,len(cigale['help_id'])):
    cigale['help_id'][i]=cigale['help_id'][i].strip()
               

## Read in photoz

In [49]:
photoz=Table.read('../../dmu24/dmu24_GAMA-12/data/master_catalogue_gama-12_20171210_photoz_20180410_r_optimised.fits')

In [50]:
photoz

help_id,RA,DEC,id,z1_median,z1_min,z1_max,z1_area,z2_median,z2_min,z2_max,z2_area,za_hb,chi_r_eazy,chi_r_atlas,chi_r_cosmos,chi_r_stellar,stellar_type
str28,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str6
HELP_J115809.625-010119.203,179.540104115,-1.02200083733,415493,0.6959,0.2271,1.1167,0.776,1.7302,1.6579,1.8051,0.023,0.716334851089,0.002686655,0.0150886325,0.0201354525,0.3918725,m3iii
HELP_J115850.554-004148.083,179.710640849,-0.6966898527,415494,0.8684,0.4865,1.214,0.752,1.7465,1.6739,1.822,0.043,0.872100152911,0.04407825,0.02533725,0.043383625,3.34674,m4iii
HELP_J114948.465+001147.728,177.451936426,0.196591224527,415495,0.887,0.4999,1.2677,0.724,1.6913,1.595,1.7884,0.07,0.87771645337,0.35133575,0.08987975,0.029299375,4.303075,m4v
HELP_J120125.808-000108.647,180.357532881,-0.0190684827784,415496,0.9072,0.583,1.2542,0.75,1.6694,1.6106,1.7305,0.043,0.894666650493,0.0040718675,0.0216934375,0.028596175,1.58375425,m4iii
HELP_J120150.572+005703.469,180.460717674,0.950963732611,415497,1.6554,1.4222,1.822,0.799,-99.0,-99.0,-99.0,-99.0,1.65789758247,1.186299,4.031635,2.62545,4.5122975,m7iii
HELP_J120422.567-005913.736,181.09402893,-0.987148769669,415498,0.8732,0.5548,1.2474,0.702,1.5987,1.4735,1.7224,0.091,0.888999651538,0.00577969,0.25491375,0.28173975,0.54081825,m4iii
HELP_J115939.276+003916.795,179.913648297,0.654665172133,415499,0.7295,0.1838,1.3019,0.615,1.9048,1.595,2.2389,0.183,0.716334851089,0.00098887325,0.0336144,0.04352285,1.25826925,rk3iii
HELP_J115154.051+010414.748,177.975212106,1.0707633185,415500,0.9787,0.6508,1.3088,0.655,1.6778,1.5335,1.822,0.137,0.993648875848,0.199398525,0.020380665,0.048394,4.22107,m5iii
HELP_J120457.625+010258.007,181.240105818,1.04944633237,415501,0.5947,0.1284,1.2542,0.709,1.8528,1.6819,2.0323,0.087,0.395846438326,0.006772955,0.0390784,0.2812145,0.90032575,rk0iii


## Join CIGALE and photoz tables

In [51]:
prior=join(cigale,photoz,keys='help_id')

In [52]:
prior

help_id,bayes.dust.luminosity,bayes.dust.luminosity_err,best.omegacam_u,best.omegacam_g,best.suprime_g,best.decam_g,best.gpc1_g,best.suprime_r,best.gpc1_r,best.omegacam_r,best.decam_r,best.gpc1_i,best.omegacam_i,best.suprime_i,best.gpc1_z,best.vista_z,best.suprime_z,best.decam_z,best.gpc1_y,best.suprime_y,best.vista_y,best.ukidss_y,best.ukidss_j,best.vista_j,best.ukidss_h,best.vista_h,best.vista_ks,best.ukidss_k,RA,DEC,id,z1_median,z1_min,z1_max,z1_area,z2_median,z2_min,z2_max,z2_area,za_hb,chi_r_eazy,chi_r_atlas,chi_r_cosmos,chi_r_stellar,stellar_type
,,,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,,,,,,,,,,,,,,,,,
str28,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str6
HELP_J112916.236+004455.105,9.65287793768e+37,1.17666784092e+38,0.000980295023014,0.00152945692592,0.00155522672771,0.00156031374351,0.0015811572678,0.00319442648463,0.00323499658544,0.00344452392723,0.00365353497765,0.00506550119885,0.00507429427253,0.00516587092685,0.00571054914042,0.005799151538,0.00589362525503,0.00606803848146,0.00634927742142,0.00641461877416,0.00659292795451,0.00665987676509,0.00809632604272,0.00811067438847,0.0106524893003,0.0106792419785,0.0122238062342,0.0124163888182,172.317651842,0.748640313787,9401913,0.5866,0.1732,1.0359,0.8,-99.0,-99.0,-99.0,-99.0,0.527093516722,-99.0,-99.0,-99.0,-99.0,
HELP_J112916.735+004525.955,4.99604151924e+38,6.01405625316e+38,0.00228961737982,0.00416181870533,0.00425796740958,0.00427614671171,0.00434788511322,0.00959801620012,0.00965743796182,0.0103856772571,0.0110859039015,0.0168555632412,0.0170860313911,0.0175084982515,0.0223159250776,0.0228707747602,0.0235845969178,0.0248448303069,0.027143746692,0.027827712213,0.0299828410087,0.0306283344946,0.0437204990637,0.0439311275299,0.0693150570334,0.0696774039528,0.0906166212251,0.0936494380324,172.319729599,0.757209664906,10715740,0.5535,0.2162,0.9175,0.795,-99.0,-99.0,-99.0,-99.0,0.442606570742,0.0850949,0.0989438666667,0.0923512666667,9.717435,m3ii
HELP_J112916.868+004538.834,6.74395178932e+35,2.11732503197e+35,0.00797875041146,0.0298080393801,0.0305873658148,0.0307507600584,0.0316232736597,0.0574327371112,0.0577872312662,0.0598883825059,0.0617535017724,0.0808511378306,0.081672435631,0.0833186989953,0.101873791141,0.104195668445,0.106982286305,0.114021556098,0.125815868503,0.12929421768,0.137854161055,0.14010272913,0.163531125537,0.163858604646,0.207051791153,0.208521374093,0.195515497984,0.19135190687,172.320283365,0.760787329227,9485122,0.0486,0.001,0.0984,0.788,-99.0,-99.0,-99.0,-99.0,0.0501443637558,0.215585285714,0.608331428571,2.41235428571,1.37624242857,rk1iii
HELP_J112916.993+004528.665,1.06941218163e+37,1.18786921122e+37,0.000958880860804,0.00204094231687,0.00212978078874,0.00215053957163,0.00221212661902,0.00438028344746,0.00439633094758,0.00453091385133,0.00465871777647,0.00602217800415,0.00607213083905,0.0061811261165,0.00718156074182,0.00729981335821,0.00744050683883,0.00781017044537,0.00848844401419,0.00862261078026,0.00902709450123,0.00914284994064,0.012212649293,0.0122324762736,0.0145243616455,0.0145698906336,0.0179420683456,0.0178088588752,172.320805757,0.757962388233,10222287,0.2516,0.0628,0.482,0.805,-99.0,-99.0,-99.0,-99.0,0.198091697744,0.0029134775,0.01379993,0.13480295,0.14214465,m3v
HELP_J112917.687+004454.217,8.27142268505e+37,5.40913585177e+37,0.00137585337232,0.00538198184203,0.0058148696356,0.00590330344897,0.00614237388744,0.0182970759978,0.0183700985544,0.0194974307305,0.0205248944548,0.0319632835879,0.0324309746237,0.0334546451974,0.0437975562519,0.0447575259366,0.0461367954266,0.0498327120644,0.0556160601284,0.0580288181608,0.0628588008841,0.0640299734127,0.098059365204

In [53]:
from astropy.cosmology import Planck15 as cosmo
from astropy import units as u
f_pred=prior['bayes.dust.luminosity']/(4*np.pi*cosmo.luminosity_distance(prior['z1_median']).to(u.cm))


In [54]:
prior=prior[np.isfinite(f_pred.value)][np.log10(f_pred.value[np.isfinite(f_pred.value)])>8.5]

In [55]:
prior['DEC'].name='Dec'

## Read in Maps

In [56]:

pswfits='../../dmu19/dmu19_HELP-SPIRE-maps/data/GAMA-12_SPIRE250_v0.9.fits'#SPIRE 250 map
pmwfits='../../dmu19/dmu19_HELP-SPIRE-maps/data/GAMA-12_SPIRE350_v0.9.fits'#SPIRE 350 map
plwfits='../../dmu19/dmu19_HELP-SPIRE-maps/data/GAMA-12_SPIRE500_v0.9.fits'#SPIRE 500 map

#output folder
output_folder='./'

In [57]:
from astropy.io import fits
from astropy import wcs

#-----250-------------
hdulist = fits.open(pswfits)
im250phdu=hdulist[0].header
im250hdu=hdulist[1].header

im250=hdulist[1].data*1.0E3 #convert to mJy
nim250=hdulist[3].data*1.0E3 #convert to mJy
w_250 = wcs.WCS(hdulist[1].header)
pixsize250=3600.0*w_250.wcs.cdelt #pixel size (in arcseconds)
hdulist.close()
#-----350-------------
hdulist = fits.open(pmwfits)
im350phdu=hdulist[0].header
im350hdu=hdulist[1].header

im350=hdulist[1].data*1.0E3 #convert to mJy
nim350=hdulist[3].data*1.0E3 #convert to mJy
w_350 = wcs.WCS(hdulist[1].header)
pixsize350=3600.0*w_350.wcs.cdelt #pixel size (in arcseconds)
hdulist.close()
#-----500-------------
hdulist = fits.open(plwfits)
im500phdu=hdulist[0].header
im500hdu=hdulist[1].header
im500=hdulist[1].data*1.0E3 #convert to mJy
nim500=hdulist[3].data*1.0E3 #convert to mJy
w_500 = wcs.WCS(hdulist[1].header)
pixsize500=3600.0*w_500.wcs.cdelt #pixel size (in arcseconds)
hdulist.close()

In [58]:
## Set XID+ prior class

In [59]:
#---prior250--------
prior250=xidplus.prior(im250,nim250,im250phdu,im250hdu, moc=Sel_func)#Initialise with map, uncertianty map, wcs info and primary header
prior250.prior_cat(prior['RA'] ,prior['Dec'] ,'GAMA12_Ldust_prediction_results.fits',ID=prior['help_id'] )#Set input catalogue
prior250.prior_bkg(-5.0,5)#Set prior on background (assumes Gaussian pdf with mu and sigma)
#---prior350--------
prior350=xidplus.prior(im350,nim350,im350phdu,im350hdu, moc=Sel_func)
prior350.prior_cat(prior['RA'] ,prior['Dec'] ,'GAMA12_Ldust_prediction_results.fits',ID=prior['help_id'] )
prior350.prior_bkg(-5.0,5)

#---prior500--------
prior500=xidplus.prior(im500,nim500,im500phdu,im500hdu, moc=Sel_func)
prior500.prior_cat(prior['RA'] ,prior['Dec'] ,'GAMA12_Ldust_prediction_results.fits',ID=prior['help_id'] )
prior500.prior_bkg(-5.0,5)

In [60]:
#pixsize array (size of pixels in arcseconds)
pixsize=np.array([pixsize250,pixsize350,pixsize500])
#point response function for the three bands
prfsize=np.array([18.15,25.15,36.3])
#use Gaussian2DKernel to create prf (requires stddev rather than fwhm hence pfwhm/2.355)
from astropy.convolution import Gaussian2DKernel

##---------fit using Gaussian beam-----------------------
prf250=Gaussian2DKernel(prfsize[0]/2.355,x_size=101,y_size=101)
prf250.normalize(mode='peak')
prf350=Gaussian2DKernel(prfsize[1]/2.355,x_size=101,y_size=101)
prf350.normalize(mode='peak')
prf500=Gaussian2DKernel(prfsize[2]/2.355,x_size=101,y_size=101)
prf500.normalize(mode='peak')

pind250=np.arange(0,101,1)*1.0/pixsize[0,1] #get 250 scale in terms of pixel scale of map
pind350=np.arange(0,101,1)*1.0/pixsize[1,1] #get 350 scale in terms of pixel scale of map
pind500=np.arange(0,101,1)*1.0/pixsize[2,1] #get 500 scale in terms of pixel scale of map

prior250.set_prf(prf250.array,pind250,pind250)#requires psf as 2d grid, and x and y bins for grid (in pixel scale)
prior350.set_prf(prf350.array,pind350,pind350)
prior500.set_prf(prf500.array,pind500,pind500)

In [61]:
import pickle
#from moc, get healpix pixels at a given order
from xidplus import moc_routines
order=9
tiles=moc_routines.get_HEALPix_pixels(order,prior250.sra,prior250.sdec,unique=True)
order_large=6
tiles_large=moc_routines.get_HEALPix_pixels(order_large,prior250.sra,prior250.sdec,unique=True)
print('----- There are '+str(len(tiles))+' tiles required for input catalogue and '+str(len(tiles_large))+' large tiles')
output_folder='./'
outfile=output_folder+'Master_prior.pkl'
xidplus.io.pickle_dump({'priors':[prior250,prior350,prior500],'tiles':tiles,'order':order,'version':xidplus.io.git_version()},outfile)
outfile=output_folder+'Tiles.pkl'
with open(outfile, 'wb') as f:
    pickle.dump({'tiles':tiles,'order':order,'tiles_large':tiles_large,'order_large':order_large,'version':xidplus.io.git_version()},f)
raise SystemExit()

----- There are 5013 tiles required for input catalogue and 100 large tiles
writing total_bytes=1493072309...
writing bytes [0, 1073741824)... done.
writing bytes [1073741824, 1493072309)... done.


SystemExit: 

In [ ]:
prior250.nsrc